# Rabi measurement and pulsed ODMR

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [2]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')    

In [3]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [4]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV1'
setup['poi']='poi_20170908_1920_43_092302'
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 40000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [5]:
# for pulsed ODMR:
odmr_dict = OrderedDict()
odmr_dict['pi_length'] = 123e-9
odmr_dict['freq_res'] = 3e+5
odmr_dict['points'] = 100
odmr_dict['measurement_time'] = 20
odmr_dict['fit_function']='Pulsed ODMR'

mw_freq = 1368.16e+6 # in MHz
mw_power = -13

freq =  (mw_freq-0.1e+9)-odmr_dict['points']*odmr_dict['freq_res']/2

pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(freq)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()

In [9]:
mw_freq = 1608.9e+6 # in MHz
mw_power = -9
pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(mw_freq-0.1e+9)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()

In [33]:
# for Rabi:
rabi = OrderedDict()
rabi['mw_amp'] = 0.111
rabi['mw_freq'] = 100.0e6 
rabi['tau_start'] = 300.0e-9
rabi['tau_step'] =  25.0e-9
rabi['number_of_taus'] = 100
rabi['measurement_time'] = 60
rabi['refocus_interval'] = 2*60

In [31]:
setup['poi'] = poimanagerlogic.active_poi.get_key()
refocus_poi(setup['poi'])

2D gaussian fit not successfull
118340.0 0.0


True

In [34]:
do_rabi()

(False, 5.2306788674027365e-07, 28.522823671285114, 1911797.7328563172, 0.81606813291014346)

In [35]:
pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True, False)

'G:\\\\data\\\\QudiData\\2018\\07\\20180711\\PulsedMeasurement'

In [6]:
def do_rabi(generate_new=True, save_tag='', load_tag=''):
    
    length = rabi['number_of_taus'] * rabi['tau_step']
            
    # sanity check for long sequences:
    if length > 3e-3:
        print("Rabi exceeds 3 ms too long for AWG5014C")
        return True,0,0,0,0
    # generate ensemble object
#     rabi['tau_res'] = rabi['length'] / (rabi['points'] - 1)
    tau_arr = np.arange(rabi['number_of_taus']) * rabi['tau_step']+rabi['tau_start']
    if generate_new:
        sequencegeneratorlogic.delete_ensemble('Rabi')
        sequencegeneratorlogic.generate_rabi('Rabi', rabi['tau_start'], rabi['tau_step'], rabi['number_of_taus'], rabi['mw_freq'], 
                                        rabi['mw_amp'], 'a_ch1', setup['laser_length'] , 1.0, setup['aom_delay'], 
                                        setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'Rabi' not in sequencegeneratorlogic.saved_pulse_block_ensembles:
            sequencegeneratorlogic.log.error('Ensemble generation of Rabi in notebook timed out.')
            return
        pulsedmasterlogic.sample_block_ensemble('Rabi', True)
    else:
        pulsedmasterlogic.load_asset_into_channels('Rabi')
    while pulsedmasterlogic.status_dict['sauplo_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmasterlogic.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, rabi['number_of_taus'], 100e-6, [], False)
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+1e-6, rabi['number_of_taus'])
    pulsedmasterlogic.analysis_interval_changed(1)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < rabi['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.manually_pull_data() 
    time.sleep(1)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.2)
        time.sleep(2)
    # do fit
    x, y, param_dict = pulsedmeasurementlogic.do_fit('Rabi')
    new_freq = param_dict.params['frequency'].value
    contrast = 2.*param_dict.params['amplitude'].value
    offset = param_dict.params['offset'].value
    period = 1./new_freq
    
#     # save data
#     if save_tag is None:
#         pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)
#     else:
#         pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)
    
    # write logfile
    write_to_logfile('Rabi_log', str(datetime.datetime.now()),name= setup['NV_name'], amplitude=rabi['mw_amp'], period=period,
                     frequency=new_freq, contrast=contrast, offset= offset)
    return user_terminated, period, contrast*100, new_freq, offset

In [9]:
def do_odmr(generate_new=True,save_data=False):
    # generate ensemble object
    if odmr_dict['pi_length']>3e-3:
        print("Pulsed ODMR exceeds length of 3ms")
        return
        
    freq_arr = (mw_freq-odmr_dict['points']*odmr_dict['freq_res']/2) + np.arange(odmr_dict['points']) * odmr_dict['freq_res']    
        
    if generate_new:
        sequencegeneratorlogic.delete_ensemble('PulsedODMR')
        sequencegeneratorlogic.generate_pulsed_odmr('PulsedODMR', odmr_dict['pi_length'], 100.0e6, odmr_dict['freq_res'], 
                                              odmr_dict['points'], 0.25, 'a_ch1', setup['laser_length'] , 
                                              1.0, setup['aom_delay'], setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'PulsedODMR' not in sequencegeneratorlogic.saved_pulse_block_ensembles:
            sequencegeneratorlogic.log.error('Ensemble generation of PulsedODMR from notebook did not work.')
            return
        pulsedmasterlogic.sample_block_ensemble('PulsedODMR', True)
    else:
        pulsedmasterlogic.load_asset_into_channels('PulsedODMR')
    while pulsedmasterlogic.status_dict['sauplo_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(0.5)
    # delete old fit
    pulsedmasterlogic.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmasterlogic.measurement_sequence_settings_changed(freq_arr, odmr_dict['points'], 100e-6, [], False)
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.2e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    # perform measurement
    pulsedmasterlogic.start_measurement()
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < odmr_dict['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(1)
    pulsedmasterlogic.manually_pull_data() 
    time.sleep(1)
    pulsedmasterlogic.stop_measurement()
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
#    pulsedmaster.save_measurement_data('Hz','PulsedODMR'+setup['NV_name'])
    time.sleep(2)
    if odmr_dict['fit_function']=='Pulsed ODMR':
        x, y, param_dict= pulsedmeasurementlogic.do_fit('Pulsed ODMR')
        new_freq = param_dict.params['center'].value
        contrast = param_dict.params['contrast'].value
        linewidth = param_dict.params['fwhm'].value
            
    if save_data:
        pulsedmasterlogic.save_measurement_data('Hz','ODMR_'+setup['NV_name'],True)
    # write logfile
    write_to_logfile('PulsedODMR_log', str(datetime.datetime.now()),name= setup['NV_name'], frequency=new_freq,
                     contrast=contrast,linewidth=linewidth)
    return user_terminated, new_freq, contrast, linewidth

In [40]:
do_odmr()

(False, 1374669168.3132148, -6.2424458658378565, 1134429.0873497562)

In [9]:
def refocus_poi(poi=None, max_drift=0.5, min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.getState() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.getState() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True
def do_rabi_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    rabi_total_time = rabi['measurement_time']
    rabi_runtime = 0.0
    rabi['measurement_time'] = rabi['refocus_interval']
    end_measure = do_rabi(True, 'rabi_refocus','')[0]
    rabi_runtime += rabi['refocus_interval']
    while rabi_total_time > rabi_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_rabi(False, 'rabi_refocus', 'rabi_refocus')[0]
        rabi_runtime += rabi['refocus_interval']
        print(rabi_runtime)
        if end_measure:
            break
    rabi['measurement_time'] = rabi_total_time
    write_to_logfile('rabi_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=rabi_runtime)
    time.sleep(2)
    return end_measure

In [101]:
setup['poi'] = poimanagerlogic.active_poi.get_key()
do_rabi_refocus(setup['poi'])

102822.0 4.217838294151262e-09
240.0
102822.0 2.0188477992466656e-08
360.0
102822.0 1.1847903645553727e-08
480.0
102822.0 1.435833759302744e-09
600.0
102822.0 2.943735344569133e-09
720.0
102822.0 5.085913480455933e-10
840.0
102822.0 6.9467061522733445e-09
960.0
102822.0 3.0328482435274277e-09
1080.0
102822.0 1.7584658497108905e-08
1200.0
102822.0 5.258200258941113e-10
1320.0
102822.0 2.6078308261895953e-09
1440.0
102822.0 1.2716049301855423e-09
1560.0
102822.0 1.6214762570867093e-08
1680.0
102822.0 2.0383019902689968e-08
1800.0
102822.0 2.120365207063579e-10
1920.0
102822.0 4.572655596101491e-09
2040.0
102822.0 4.450085645289824e-09
2160.0
102822.0 1.4133175336715248e-08
2280.0
102822.0 1.3217873570423402e-08
2400.0
102822.0 1.515454457482932e-08
2520.0
102822.0 1.5413117518132523e-09
2640.0
2D gaussian fit not successfull
102822.0 0.0
2760.0
102822.0 1.680851099042477e-08
2880.0
102822.0 2.105654879531484e-08
3000.0
2D gaussian fit not successfull
102822.0 0.0
3120.0
102822.0 2.365620

False